---

<center><h1> Spark ML Assignment </h1></center>

----

A retail company “ABC Private Limited” wants to understand the customer purchase behaviour (specifically, purchase amount) against various products of different categories.
They have shared purchase summary of various customers for selected high volume products from last month.
The data set also contains customer demographics (age, gender, marital status, city_type, stay_in_current_city),
product details (product_id and product category) and
Total purchase_amount from last month.

Now, they want to build a model to predict the purchase amount of customer against various products which will help them to create personalized offer for customers against different products.


So, in this notebook we will build a Regression model to predict the purchase amount of customer against various products.

----

In [0]:
# !pip3 install matplotlib --user

In [0]:
# importing the required libraries
from pyspark.sql import SparkSession
import pyspark.sql.types as tp
from pyspark.sql import functions as F
import pandas as pd

from pyspark.ml.feature import MinMaxScaler

# libraries to make plots
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
# create spark session
spark = SparkSession.builder.getOrCreate()

In [0]:
# reading the train data
train_data = spark.read.csv("/FileStore/tables/train_Spark_ML_Assignment.csv",inferSchema=True, header=True)

# reading the test data
test_data  = spark.read.csv("/FileStore/tables/test_Spark_ML_Assignment.csv", inferSchema=True, header=True)

In [0]:
train_data.count() # number of rows

Out[7]: 550068

In [0]:
train_data.columns # columns

Out[8]: ['User_ID',
 'Product_ID',
 'Gender',
 'Age',
 'Occupation',
 'City_Category',
 'Stay_In_Current_City_Years',
 'Marital_Status',
 'Product_Category_1',
 'Product_Category_2',
 'Product_Category_3',
 'Purchase']

---

#### `READING THE DATASET`

---

 * **`User_ID`**:  User ID 
 * **`Product_ID`**:   Product ID
 * **`Gender`**:   Sex of user
 * **`Age`**:  Age in Bins
 * **`Occupation`**:  Occupation (Masked)
 * **`City_category`**:  Category of the city (A,B,C)
 * **`Stay_In_Current_City_Years`**:  Number of Years stay in current city
 * **`Marital_Status`**: Marital status
 * **`Product_Category_1`**:  Product Category (Masked)
 * **`Product_Category_2`**: Product Category may belongs to other category also (masked)
 * **`Product_Category_3`**:  Product Category may belongs to other category also (masked)
 * **`Purchase`**:  Purchase Amount( **`Target Variable`** )

---

In [0]:
# data type of the columns
train_data.printSchema()

root
-- User_ID: integer (nullable = true)
-- Product_ID: string (nullable = true)
-- Gender: string (nullable = true)
-- Age: string (nullable = true)
-- Occupation: integer (nullable = true)
-- City_Category: string (nullable = true)
-- Stay_In_Current_City_Years: string (nullable = true)
-- Marital_Status: integer (nullable = true)
-- Product_Category_1: integer (nullable = true)
-- Product_Category_2: integer (nullable = true)
-- Product_Category_3: integer (nullable = true)
-- Purchase: integer (nullable = true)

---

**`Numerical`**
- User_ID
- Occupation
- Marital_Status
- Product_Category_1
- Product_Category_2
- Product_Category_3
- Purchase

**`Categorical`**
- Product_ID
- Gender
- Age
- City_Category
- Stay_In_Current_City_Years


---

The `Target Variable` for our use-case is `Purchase`.

#### SQL Querying

In [0]:
# Create a view
train_data.createOrReplaceTempView('train_table')

Questions:
1. Average Purchase amount?
2. Counting and Removing null values
3. How many distinct values per column?
4. Count category values within each of the following column:
● Gender
● Age
● City_Category
● Stay_In_Current_City_Years
● Marital_Status
5. Calculate average Purchase for each of the following columns:
● Gender
● Age
● City_Category
● Stay_In_Current_City_Years
● Marital_Status
6. Label encode the following columns:
● Age
● Gender
● Stay_In_Current_City_Years
● City_Category
7. One-Hot encode following columns:
● Gender
● City_Category
● Occupation
8. Build a baseline model using any of the ML algorithms.
9. Model improvement with Grid-Search CV
10. Create a Spark ML Pipeline for the final model.

1. Average Purchase amount?

In [0]:
%python

Avg_Purchase_Amount = spark.sql("""

SELECT round(avg(Purchase),2) as Avg_Purchase_Amount
FROM train_table 

""")

Avg_Purchase_Amount.show()

+-------------------+
Avg_Purchase_Amount|
+-------------------+
 9263.97|
+-------------------+

Avg_Purchase_Amount = 9263.97

---------

2.Counting and Removing null values

In [0]:
### null values in each column
for c in train_data.columns:
    # define the condition
    missing_values = F.isnull(c)
    
    # filter the data with condition and count the number of data points
    Count_of_missing_values = train_data.filter(missing_values).count()
    
    # print the result
    print(c, Count_of_missing_values)

User_ID 0
Product_ID 0
Gender 0
Age 0
Occupation 0
City_Category 0
Stay_In_Current_City_Years 0
Marital_Status 0
Product_Category_1 0
Product_Category_2 173638
Product_Category_3 383247
Purchase 0

---

Count of null values on:

 * **`Product_Category_2`**:   173638
 * **`Product_Category_3`**:   383247


---

###### Removing null values

In [0]:
train_data = train_data.dropna()

In [0]:
# Verifying it

### null values in each column
for c in train_data.columns:
    # define the condition
    missing_values = F.isnull(c)
    
    # filter the data with condition and count the number of data points
    Count_of_missing_values = train_data.filter(missing_values).count()
    
    # print the result
    print(c, Count_of_missing_values)

User_ID 0
Product_ID 0
Gender 0
Age 0
Occupation 0
City_Category 0
Stay_In_Current_City_Years 0
Marital_Status 0
Product_Category_1 0
Product_Category_2 0
Product_Category_3 0
Purchase 0

---------

3.How many distinct values per column?

In [0]:
# distinct values in each column
distinctcols = train_data.agg(*(F.countDistinct(F.col(c)).alias(c) for c in train_data.columns))
distinctDF = pd.DataFrame(distinctcols.collect(), columns= distinctcols.schema.names)
distinctDF.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,5870,528,2,7,21,3,5,2,12,14,15,13876


---

 - There are 5870 unique values in the `User_ID` column.
 - There are 528 unique values in the `Product_ID` column.
 - Gender has 2 distinct values, age has 7, City_Category has 3,  Stay_In_Current_City_Years has 5, marital_status has 5
 - Occupation has 21 unique values, Product_Category_1 has 12, Product_Category_2 has 14, Product_Category_3 has 15
---

---

4.Count category values within each of the following column:

● Gender
● Age
● City_Category
● Stay_In_Current_City_Years
● Marital_Status

In [0]:
# number of datapoints with each gender
top_gender = train_data.groupBy("Gender").agg(F.count("Gender").alias("gender_count"), F.round((F.count("Gender")/train_data.count())*100, 4).alias("percentage"))

top_gender.show()

+------+------------+----------+
Gender|gender_count|percentage|
+------+------------+----------+
 F| 37594| 22.5355|
 M| 129227| 77.4645|
+------+------------+----------+

In [0]:
# number of datapoints with each Age Bins
top_Age_bins = train_data.groupBy("Age").agg(F.count("Age").alias("Age_bin_count"), F.round((F.count("Age")/train_data.count())*100, 4).alias("percentage"))


top_Age_bins.show()

+-----+-------------+----------+
 Age|Age_bin_count|percentage|
+-----+-------------+----------+
18-25| 31316| 18.7722|
26-35| 66942| 40.128|
 0-17| 4873| 2.9211|
46-50| 13374| 8.017|
51-55| 11166| 6.6934|
36-45| 33285| 19.9525|
 55+| 5865| 3.5157|
+-----+-------------+----------+

In [0]:
# number of datapoints with each City_Category
top_City_Category = train_data.groupBy("City_Category").agg(F.count("City_Category").alias("City_Category_count"), F.round((F.count("City_Category")/train_data.count())*100, 4).alias("percentage"))

top_City_Category.show()

+-------------+-------------------+----------+
City_Category|City_Category_count|percentage|
+-------------+-------------------+----------+
 B| 69243| 41.5074|
 C| 56059| 33.6043|
 A| 41519| 24.8884|
+-------------+-------------------+----------+

In [0]:
# number of datapoints with each Stay_In_Current_City_Years
top_Stay_In_Current_City_Years = train_data.groupBy("Stay_In_Current_City_Years").agg(F.count("Stay_In_Current_City_Years").alias("Stay_In_Current_City_Years_count"), F.round((F.count("Stay_In_Current_City_Years")/train_data.count())*100, 4).alias("percentage"))

top_Stay_In_Current_City_Years.show()

+--------------------------+--------------------------------+----------+
Stay_In_Current_City_Years|Stay_In_Current_City_Years_count|percentage|
+--------------------------+--------------------------------+----------+
 3| 29268| 17.5446|
 0| 22389| 13.421|
 4+| 25362| 15.2031|
 1| 58287| 34.9398|
 2| 31515| 18.8915|
+--------------------------+--------------------------------+----------+

In [0]:
# number of datapoints with each Marital_Status
top_Marital_Status = train_data.groupBy("Marital_Status").agg(F.count("Marital_Status").alias("Marital_Status_count"), F.round((F.count("Marital_Status")/train_data.count())*100, 4).alias("percentage"))

top_Marital_Status.show()

+--------------+--------------------+----------+
Marital_Status|Marital_Status_count|percentage|
+--------------+--------------------+----------+
 1| 67202| 40.2839|
 0| 99619| 59.7161|
+--------------+--------------------+----------+

---

5.Calculate average Purchase for each of the following columns:


● Gender
● Age
● City_Category
● Stay_In_Current_City_Years
● Marital_Status

In [0]:
%python

# Gender

Avg_Purchase_Amount_Gender = spark.sql("""

SELECT Gender, avg(Purchase) as Avg_Purchase_Amount_Gender
FROM train_table 
group by Gender

""")

Avg_Purchase_Amount_Gender.show()

+------+--------------------------+
Gender|Avg_Purchase_Amount_Gender|
+------+--------------------------+
 F| 8734.565765155476|
 M| 9437.526040472265|
+------+--------------------------+

In [0]:
%python

# Age

Avg_Purchase_Amount_Age = spark.sql("""

SELECT Age, avg(Purchase) as Avg_Purchase_Amount_Age
FROM train_table 
group by Age
order by Age

""")

Avg_Purchase_Amount_Age.show()

+-----+-----------------------+
 Age|Avg_Purchase_Amount_Age|
+-----+-----------------------+
 0-17| 8933.464640444974|
18-25| 9169.663606261289|
26-35| 9252.690632869888|
36-45| 9331.350694917874|
46-50| 9208.625697468327|
51-55| 9534.808030960236|
 55+| 9336.280459449405|
+-----+-----------------------+

In [0]:
%python

# City_Category

Avg_Purchase_Amount_City_Category = spark.sql("""

SELECT City_Category, avg(Purchase) as Avg_Purchase_Amount_City_Category
FROM train_table 
group by City_Category
order by City_Category

""")

Avg_Purchase_Amount_City_Category.show()


+-------------+---------------------------------+
City_Category|Avg_Purchase_Amount_City_Category|
+-------------+---------------------------------+
 A| 8911.939216084484|
 B| 9151.300562781986|
 C| 9719.92099313568|
+-------------+---------------------------------+

In [0]:

Avg_Purchase_Amount_City = train_data.groupBy("City_Category").agg(F.avg("Purchase").alias("Avg_Purchase_Amount_City_Category"))
Avg_Purchase_Amount_City.show()

+-------------+---------------------------------+
City_Category|Avg_Purchase_Amount_City_Category|
+-------------+---------------------------------+
 B| 11488.090521785596|
 C| 12207.51699102731|
 A| 11199.868782966834|
+-------------+---------------------------------+

In [0]:
%python

# Stay_In_Current_City_Years

Avg_Purchase_Amount_Stay_In_Current_City_Years = spark.sql("""

SELECT Stay_In_Current_City_Years, avg(Purchase) as Avg_Purchase_Amount_Stay_In_Current_City_Years
FROM train_table 
group by Stay_In_Current_City_Years
order by Stay_In_Current_City_Years

""")

Avg_Purchase_Amount_Stay_In_Current_City_Years.show()


+--------------------------+----------------------------------------------+
Stay_In_Current_City_Years|Avg_Purchase_Amount_Stay_In_Current_City_Years|
+--------------------------+----------------------------------------------+
 0| 9180.075122987177|
 1| 9250.145923300364|
 2| 9320.429810090536|
 3| 9286.904119221284|
 4+| 9275.59887165687|
+--------------------------+----------------------------------------------+

In [0]:
%python

# Marital_Status

Avg_Purchase_Amount_Marital_Status = spark.sql("""

SELECT Marital_Status, avg(Purchase) as Avg_Purchase_Amount_Marital_Status
FROM train_table 
group by Marital_Status
order by Marital_Status

""")

Avg_Purchase_Amount_Marital_Status.show()


+--------------+----------------------------------+
Marital_Status|Avg_Purchase_Amount_Marital_Status|
+--------------+----------------------------------+
 0| 9265.907618921507|
 1| 9261.174574082374|
+--------------+----------------------------------+

---

6.Label encode the following columns:

● Age
● Gender
● Stay_In_Current_City_Years
● City_Category

In [0]:
# importing libraries
from pyspark.ml.feature import StringIndexer

##### Creating the StringIndexer Objects

In [0]:


# label encode Age

SI_Age = StringIndexer(inputCol= "Age", outputCol= "Age_le" , handleInvalid="skip")

# label encode Gender
SI_Gender = StringIndexer(inputCol= "Gender", outputCol= "Gender_le", handleInvalid= "skip")

# label encode Stay_In_Current_City_Years 
SI_Stay_In_Current_City_Years      = StringIndexer(inputCol= "Stay_In_Current_City_Years", outputCol= "Stay_In_Current_City_Years_le", handleInvalid= "skip")

# label encode City_Category
SI_City_Category = StringIndexer(inputCol= "City_Category", outputCol= "City_Category_le", handleInvalid="skip")

##### Fit the StringIndexer Objects

In [0]:
# label encode object Age
SI_Age_Obj = SI_Age.fit(train_data)

# label encode object Gender
SI_Gender_Obj = SI_Gender.fit(train_data)

# label encode object Stay_In_Current_City_Years
SI_Stay_In_Current_City_Years_Obj = SI_Stay_In_Current_City_Years.fit(train_data)

# label encode object City_Category
SI_City_Category_Obj = SI_City_Category.fit(train_data)

##### Transform the StringIndexer Objects

In [0]:
# label encode Age
train_data_encoded = SI_Age_Obj.transform(train_data)

# label encode Gender
train_data_encoded = SI_Gender_Obj.transform(train_data_encoded)

# label encode object Stay_In_Current_City_Years
train_data_encoded = SI_Stay_In_Current_City_Years_Obj.transform(train_data_encoded)

# label encode object City_Category
train_data_encoded = SI_City_Category_Obj.transform(train_data_encoded)



----

###### `Make the Transformation on the test data`

---

In [0]:
# label encode Age
test_data_encoded = SI_Age_Obj.transform(test_data)

# label encode Gender
test_data_encoded = SI_Gender_Obj.transform(test_data_encoded)

# label encode object Stay_In_Current_City_Years
test_data_encoded = SI_Stay_In_Current_City_Years_Obj.transform(test_data_encoded)

# label encode object City_Category
test_data_encoded = SI_City_Category_Obj.transform(test_data_encoded)



##### Label encoded columns ● Age ● Gender ● Stay_In_Current_City_Years ● City_Category

In [0]:
train_data_encoded.select('Age_le',
 'Gender_le',
 'Stay_In_Current_City_Years_le',
 'City_Category_le').show()

+------+---------+-----------------------------+----------------+
Age_le|Gender_le|Stay_In_Current_City_Years_le|City_Category_le|
+------+---------+-----------------------------+----------------+
 6.0| 1.0| 1.0| 2.0|
 3.0| 0.0| 1.0| 0.0|
 0.0| 0.0| 0.0| 2.0|
 4.0| 1.0| 0.0| 2.0|
 4.0| 1.0| 0.0| 2.0|
 1.0| 0.0| 0.0| 0.0|
 0.0| 0.0| 3.0| 1.0|
 0.0| 0.0| 3.0| 1.0|
 0.0| 0.0| 4.0| 1.0|
 1.0| 1.0| 3.0| 0.0|
 1.0| 1.0| 3.0| 0.0|
 1.0| 1.0| 3.0| 0.0|
 1.0| 1.0| 3.0| 0.0|
 1.0| 1.0| 3.0| 0.0|
 1.0| 1.0| 3.0| 0.0|
 1.0| 1.0| 3.0| 0.0|
 1.0| 1.0| 3.0| 0.0|
 0.0| 1.0| 0.0| 1.0|
 3.0| 0.0| 2.0| 1.0|
 0.0| 0.0| 0.0| 2.0|
+------+---------+-----------------------------+----------------+
only showing top 20 rows

In [0]:
test_data_encoded.select('Age_le',
 'Gender_le',
 'Stay_In_Current_City_Years_le',
 'City_Category_le').show()

+------+---------+-----------------------------+----------------+
Age_le|Gender_le|Stay_In_Current_City_Years_le|City_Category_le|
+------+---------+-----------------------------+----------------+
 3.0| 0.0| 1.0| 0.0|
 0.0| 0.0| 4.0| 1.0|
 1.0| 1.0| 3.0| 0.0|
 1.0| 1.0| 3.0| 0.0|
 0.0| 1.0| 0.0| 1.0|
 3.0| 0.0| 2.0| 1.0|
 3.0| 0.0| 2.0| 1.0|
 3.0| 0.0| 2.0| 1.0|
 0.0| 0.0| 0.0| 2.0|
 2.0| 0.0| 3.0| 2.0|
 0.0| 0.0| 1.0| 0.0|
 0.0| 0.0| 1.0| 0.0|
 0.0| 0.0| 1.0| 0.0|
 0.0| 0.0| 1.0| 0.0|
 0.0| 1.0| 1.0| 1.0|
 1.0| 0.0| 0.0| 1.0|
 3.0| 0.0| 0.0| 2.0|
 3.0| 0.0| 0.0| 2.0|
 2.0| 1.0| 4.0| 2.0|
 3.0| 0.0| 3.0| 1.0|
+------+---------+-----------------------------+----------------+
only showing top 20 rows

---

7.One-Hot encode following columns:


● Gender
● City_Category
● Occupation

In [0]:
from pyspark.ml.feature import OneHotEncoder


In [0]:
OHE_train = OneHotEncoder(inputCols=["Gender_le",
                                     "City_Category_le",
                                      "Occupation"],
                          outputCols=["Gender_ohe",
                                      "City_Category_ohe",
                                      "Occupation_ohe"])


Fit & Transform the data.

In [0]:
# OHE object
OHE_Obj = OHE_train.fit(train_data_encoded)

In [0]:
# Transform train data
train_data_encoded = OHE_Obj.transform(train_data_encoded)

In [0]:
# view the one hot encoded data
train_data_encoded.select("Gender_ohe",
                          "City_Category_ohe", 
                          "Occupation_ohe").show()
                                             

+-------------+-----------------+---------------+
 Gender_ohe|City_Category_ohe| Occupation_ohe|
+-------------+-----------------+---------------+
 (1,[],[])| (2,[],[])|(20,[10],[1.0])|
(1,[0],[1.0])| (2,[0],[1.0])| (20,[7],[1.0])|
(1,[0],[1.0])| (2,[],[])| (20,[],[])|
 (1,[],[])| (2,[],[])| (20,[9],[1.0])|
 (1,[],[])| (2,[],[])| (20,[9],[1.0])|
(1,[0],[1.0])| (2,[0],[1.0])| (20,[1],[1.0])|
(1,[0],[1.0])| (2,[1],[1.0])|(20,[12],[1.0])|
(1,[0],[1.0])| (2,[1],[1.0])|(20,[12],[1.0])|
(1,[0],[1.0])| (2,[1],[1.0])|(20,[17],[1.0])|
 (1,[],[])| (2,[0],[1.0])| (20,[1],[1.0])|
 (1,[],[])| (2,[0],[1.0])| (20,[1],[1.0])|
 (1,[],[])| (2,[0],[1.0])| (20,[1],[1.0])|
 (1,[],[])| (2,[0],[1.0])| (20,[1],[1.0])|
 (1,[],[])| (2,[0],[1.0])| (20,[1],[1.0])|
 (1,[],[])| (2,[0],[1.0])| (20,[1],[1.0])|
 (1,[],[])| (2,[0],[1.0])| (20,[1],[1.0])|
 (1,[],[])| (2,[0],[1.0])| (20,[1],[1.0])|
 (1,[],[])| (2,[1],[1.0])| (20,[1],[1.0])|
(1,[0],[1.0])| (2,[1],[1.0])| (20,[1],[1.0])|
(1,[0],[1.0])| (2,[],[])| (20,[7],[1.0])|
+-------------+-----------------+---------------+
only showing top 20 rows

----

###### `Make the Transformation on the test data`

---

In [0]:

# test data
test_data_encoded = OHE_Obj.transform(test_data_encoded)

In [0]:
# view the one hot encoded data
test_data_encoded.select("Gender_ohe",
                          "City_Category_ohe", 
                          "Occupation_ohe").show()
                                             

+-------------+-----------------+---------------+
 Gender_ohe|City_Category_ohe| Occupation_ohe|
+-------------+-----------------+---------------+
(1,[0],[1.0])| (2,[0],[1.0])| (20,[7],[1.0])|
(1,[0],[1.0])| (2,[1],[1.0])|(20,[17],[1.0])|
 (1,[],[])| (2,[0],[1.0])| (20,[1],[1.0])|
 (1,[],[])| (2,[0],[1.0])| (20,[1],[1.0])|
 (1,[],[])| (2,[1],[1.0])| (20,[1],[1.0])|
(1,[0],[1.0])| (2,[1],[1.0])| (20,[1],[1.0])|
(1,[0],[1.0])| (2,[1],[1.0])| (20,[1],[1.0])|
(1,[0],[1.0])| (2,[1],[1.0])| (20,[1],[1.0])|
(1,[0],[1.0])| (2,[],[])| (20,[7],[1.0])|
(1,[0],[1.0])| (2,[],[])|(20,[15],[1.0])|
(1,[0],[1.0])| (2,[0],[1.0])| (20,[7],[1.0])|
(1,[0],[1.0])| (2,[0],[1.0])| (20,[7],[1.0])|
(1,[0],[1.0])| (2,[0],[1.0])| (20,[7],[1.0])|
(1,[0],[1.0])| (2,[0],[1.0])| (20,[7],[1.0])|
 (1,[],[])| (2,[1],[1.0])| (20,[1],[1.0])|
(1,[0],[1.0])| (2,[1],[1.0])| (20,[7],[1.0])|
(1,[0],[1.0])| (2,[],[])| (20,[3],[1.0])|
(1,[0],[1.0])| (2,[],[])| (20,[3],[1.0])|
 (1,[],[])| (2,[],[])| (20,[0],[1.0])|
(1,[0],[1.0])| (2,[1],[1.0])| (20,[1],[1.0])|
+-------------+-----------------+---------------+
only showing top 20 rows

---

In [0]:
train_data_encodedCopy = train_data_encoded.select('*')
test_data_encodedCopy = test_data_encoded.select('*')

-----
----------

Feature engg 
- Product_ID and Purchase -->  1. Mean_Purchase
- Product_ID and count of products  -->  2. category_count

In [0]:
#feat Engg

# Mean_Purchase value for each product ID

Mean_Purchase_valueDF = train_data.groupby("Product_ID").agg(F.avg("Purchase").alias("Mean_Purchase"))
Mean_Purchase_valueDF.show()

+----------+------------------+
Product_ID| Mean_Purchase|
+----------+------------------+
 P00281742| 7989.982608695652|
 P00026042| 6819.873350923483|
 P00165442|10185.363636363636|
 P00209742|14638.292307692307|
 P00160042|13067.443349753694|
 P00053942|12692.453924914676|
 P00004242|11237.716814159292|
 P00127342| 12260.89497716895|
 P0094342| 2144.801652892562|
 P00019042| 4955.44|
 P00115342| 7247.309139784947|
 P00320942| 7823.077586206897|
 P00228342|12813.209302325582|
 P00136242|11103.791666666666|
 P00081142|15548.091988130564|
 P00116142|20463.791277258566|
 P00241242| 12218.58787878788|
 P0096642|10261.281609195403|
 P00129842|15348.767558528429|
 P00317642| 14620.25|
+----------+------------------+
only showing top 20 rows

In [0]:

train_data_encoded = train_data_encoded.join(Mean_Purchase_valueDF, train_data_encoded.Product_ID == Mean_Purchase_valueDF.Product_ID, "inner").drop(train_data_encoded.Product_ID)
train_data_encoded.select("Product_ID", "Mean_Purchase").show(truncate=False)


+----------+-----------------+
Product_ID|Mean_Purchase |
+----------+-----------------+
P00281742 |7989.982608695652|
P00281742 |7989.982608695652|
P00281742 |7989.982608695652|
P00281742 |7989.982608695652|
P00281742 |7989.982608695652|
P00281742 |7989.982608695652|
P00281742 |7989.982608695652|
P00281742 |7989.982608695652|
P00281742 |7989.982608695652|
P00281742 |7989.982608695652|
P00281742 |7989.982608695652|
P00281742 |7989.982608695652|
P00281742 |7989.982608695652|
P00281742 |7989.982608695652|
P00281742 |7989.982608695652|
P00281742 |7989.982608695652|
P00281742 |7989.982608695652|
P00281742 |7989.982608695652|
P00281742 |7989.982608695652|
P00281742 |7989.982608695652|
+----------+-----------------+
only showing top 20 rows

In [0]:

test_data_encoded = test_data_encoded.join(Mean_Purchase_valueDF, test_data_encoded.Product_ID == Mean_Purchase_valueDF.Product_ID, "inner").drop(test_data_encoded.Product_ID)
test_data_encoded.select("Product_ID", "Mean_Purchase").show(truncate=False)


+----------+-----------------+
Product_ID|Mean_Purchase |
+----------+-----------------+
P00281742 |7989.982608695652|
P00281742 |7989.982608695652|
P00281742 |7989.982608695652|
P00281742 |7989.982608695652|
P00281742 |7989.982608695652|
P00281742 |7989.982608695652|
P00281742 |7989.982608695652|
P00281742 |7989.982608695652|
P00281742 |7989.982608695652|
P00281742 |7989.982608695652|
P00281742 |7989.982608695652|
P00281742 |7989.982608695652|
P00281742 |7989.982608695652|
P00281742 |7989.982608695652|
P00281742 |7989.982608695652|
P00281742 |7989.982608695652|
P00281742 |7989.982608695652|
P00281742 |7989.982608695652|
P00281742 |7989.982608695652|
P00281742 |7989.982608695652|
+----------+-----------------+
only showing top 20 rows

In [0]:
#feat Engg

# Category count for each product ID
Prod_CategoryDF = train_data.groupBy("Product_ID").agg(F.count("Product_ID").alias("Product_Category_count"))
Prod_CategoryDF.show()

+----------+----------------------+
Product_ID|Product_Category_count|
+----------+----------------------+
 P00281742| 115|
 P00026042| 379|
 P00165442| 363|
 P00209742| 390|
 P00160042| 203|
 P00053942| 293|
 P00004242| 113|
 P00127342| 438|
 P0094342| 121|
 P00019042| 75|
 P00115342| 372|
 P00320942| 116|
 P00228342| 86|
 P00136242| 24|
 P00081142| 337|
 P00116142| 642|
 P00241242| 165|
 P0096642| 174|
 P00129842| 598|
 P00317642| 60|
+----------+----------------------+
only showing top 20 rows

In [0]:

train_data_encoded = train_data_encoded.join(Prod_CategoryDF, train_data_encoded.Product_ID == Prod_CategoryDF.Product_ID, "inner").drop(train_data_encoded.Product_ID)
train_data_encoded.select("Product_ID", "Product_Category_count").show(truncate=False)


+----------+----------------------+
Product_ID|Product_Category_count|
+----------+----------------------+
P00248942 |581 |
P00184942 |1440 |
P00145042 |1406 |
P00231342 |83 |
P0096642 |174 |
P00036842 |884 |
P00249542 |785 |
P00303442 |356 |
P00078742 |282 |
P00085942 |963 |
P00182642 |400 |
P00155442 |424 |
P00221342 |383 |
P00111142 |1015 |
P0094542 |174 |
P00148642 |1204 |
P00113242 |933 |
P00110842 |1281 |
P00182342 |456 |
P00042142 |388 |
+----------+----------------------+
only showing top 20 rows

In [0]:

test_data_encoded = test_data_encoded.join(Prod_CategoryDF, test_data_encoded.Product_ID == Prod_CategoryDF.Product_ID, "inner").drop(test_data_encoded.Product_ID)
test_data_encoded.select("Product_ID", "Product_Category_count").show(truncate=False)



+----------+----------------------+
Product_ID|Product_Category_count|
+----------+----------------------+
P00281742 |115 |
P00281742 |115 |
P00281742 |115 |
P00281742 |115 |
P00281742 |115 |
P00281742 |115 |
P00281742 |115 |
P00281742 |115 |
P00281742 |115 |
P00281742 |115 |
P00281742 |115 |
P00281742 |115 |
P00281742 |115 |
P00281742 |115 |
P00281742 |115 |
P00281742 |115 |
P00281742 |115 |
P00281742 |115 |
P00281742 |115 |
P00281742 |115 |
+----------+----------------------+
only showing top 20 rows

--------

In [0]:
train_data_encodedCopyModel = train_data_encoded.select('*')
test_data_encodedCopyModel = test_data_encoded.select('*')

8.Build a baseline model using any of the ML algorithms

---


#### `BaseLine MODEL BUILDING`

This is a `Regression Problem`, so we will train the data on 

 * **Linear Regression**
 
 and the Evaluation Metric is Root Mean Squared Error (RMSE)



---

In [0]:
## columns in the dataset
train_data_encoded.columns

Out[49]: ['User_ID',
 'Gender',
 'Age',
 'Occupation',
 'City_Category',
 'Stay_In_Current_City_Years',
 'Marital_Status',
 'Product_Category_1',
 'Product_Category_2',
 'Product_Category_3',
 'Purchase',
 'Age_le',
 'Gender_le',
 'Stay_In_Current_City_Years_le',
 'City_Category_le',
 'Gender_ohe',
 'City_Category_ohe',
 'Occupation_ohe',
 'Mean_Purchase',
 'Product_ID',
 'Product_Category_count']

----

So, now we will select the features and create a vector using `VectorAssembler`

 * Age_le (Label Encoded)
 * Gender_le (Label Encoded)
 * Occupation_ohe (One Hot Encoded)
 * City_Category_ohe (One Hot Encoded)
 * Stay_In_Current_City_Years_le (Label Encoded)
 * Marital_Status 
 * Product_Category_1 
 * Product_Category_2
 * Product_Category_3
 * Mean_Purchase
 * Product_Category_count

----

In [0]:
# import the library
from pyspark.ml.feature import VectorAssembler

# create feature vector
feature_vector = VectorAssembler(inputCols= ['Age_le',
                                             'Gender_le',
                                             'Occupation_ohe',
                                             'City_Category_ohe',
                                             'Stay_In_Current_City_Years_le',
                                             'Marital_Status',
                                             'Product_Category_1',
                                             'Product_Category_2',
                                             'Product_Category_3',
                                            'Mean_Purchase', 
                                             'Product_Category_count'],
                                outputCol= 'features')


In [0]:
# transform the feature vector
train_data_encoded = feature_vector.transform(train_data_encoded)

In [0]:
# transform the feature vector on test data

test_data_encoded = feature_vector.transform(test_data_encoded)

Train\Test Split

In [0]:
train_data_encoded_Split, valid_data_encoded_Split = train_data_encoded.randomSplit([0.7, 0.3])


Training the Machine Learning Algorithm

In [0]:
# importing the libraries
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

In [0]:
LR = LinearRegression(featuresCol="features", labelCol="Purchase")

In [0]:
# Fit the model

model_LR = LR.fit(train_data_encoded_Split)

print("Coefficients: " + str(model_LR.coefficients))
print("Intercept: " + str(model_LR.intercept))


Coefficients: [45.75517651577144,-20.90043926417242,254.52059126390853,-10.325493837113005,164.28084459005146,494.9597680074102,239.58932839087353,371.0447743257434,520.3554191038608,326.563572653732,274.0232266640671,432.06018951302025,132.2609774025378,164.27440160950442,416.8215262521169,4.796831716150758,357.85966058144845,442.3858024891375,434.3553419256366,467.3755806453174,271.1360014587081,-252.1621237090194,208.1619253337959,483.8545344851563,-8.605890373855516,-12.754955456939115,4.2610253590776415,0.012326223614314897,-0.23499878995548767,1.0015865831362514,-0.06772360814670914]
Intercept: -543.8079092629043

Evaluating Model Performance

In [0]:
trainingSummary = model_LR.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 3225.456858
r2: 0.597865

In [0]:
# Evaluate training data
evaluator = RegressionEvaluator(labelCol="Purchase", metricName="rmse") 

evaluator.evaluate(model_LR.transform(train_data_encoded_Split)) 

Out[58]: 3225.456858079515

In [0]:
# Evaluate validation data
evaluator.evaluate(model_LR.transform(valid_data_encoded_Split))

Out[59]: 3206.067394815539

Predicting Values on Validation data

In [0]:
predictions = model_LR.transform(valid_data_encoded_Split)


In [0]:
predictions.select("features","Purchase","prediction").show()

+--------------------+--------+------------------+
 features|Purchase| prediction|
+--------------------+--------+------------------+
(31,[0,1,12,24,26...| 11769|16474.783921772363|
(31,[0,1,12,24,26...| 2763|2559.4399312006863|
(31,[0,18,23,24,2...| 5260| 6966.369703328334|
(31,[25,26,27,28,...| 15665| 13891.67429450735|
(31,[0,1,11,26,27...| 10810|10277.184179640242|
(31,[0,1,11,26,27...| 10574|10592.890715556185|
(31,[0,1,11,26,27...| 5293| 6360.413717221881|
(31,[0,3,22,25,26...| 15447| 14323.67327033428|
(31,[0,3,22,25,26...| 11788|13466.801825394767|
(31,[14,23,24,25,...| 11927|11577.883661759253|
(31,[19,23,24,26,...| 11048| 11789.80635369286|
(31,[0,1,3,22,24,...| 12026| 11372.83740825387|
(31,[0,1,3,22,24,...| 11562| 16435.18134339421|
(31,[0,1,3,22,24,...| 12909|11052.750843154643|
(31,[0,1,3,22,24,...| 8780|6759.2754808259115|
(31,[0,3,23,24,25...| 9760|11279.453096872008|
(31,[9,26,27,28,2...| 11458|11911.661392680442|
(31,[9,26,27,28,2...| 11651|13489.776312325987|
(31,[9,26,27,28,2...| 11978|12882.916882232817|
(31,[9,26,27,28,2...| 11579|12076.341135528522|
+--------------------+--------+------------------+
only showing top 20 rows

____

9.Model improvement with Grid-Search CV

In [0]:
# import the CrossValidator and ParamGridBuilder
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [0]:
# create the ParamGridBuilder
params = ParamGridBuilder().build()

In [0]:
# create the object of the Logistic Regression Model
model_LR_CV = LinearRegression(featuresCol= "features",  labelCol="Purchase")

In [0]:
# create object of the cross validation model with numFolds = 3
cv = CrossValidator(estimator=model_LR_CV,
                    estimatorParamMaps=params,
                    evaluator=evaluator,
                    numFolds=3,
                    seed=27)

Fit the model

In [0]:
## fit the model
cv_model_LR = cv.fit(train_data_encoded_Split)

/databricks/spark/python/pyspark/ml/util.py:800: UserWarning: Cannot find mlflow module. To enable MLflow logging, install mlflow from PyPI.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

In [0]:
# Evaluate training data
evaluator = RegressionEvaluator(labelCol="Purchase", metricName="rmse") 

evaluator.evaluate(cv_model_LR.transform(train_data_encoded_Split)) 

Out[67]: 3225.456858079515

In [0]:
# evaluate model on validation data
evaluator.evaluate(cv_model_LR.transform(valid_data_encoded_Split))

Out[68]: 3206.067394815539

--Grid Search--

In [0]:
# create parameter builder

updated_params = ParamGridBuilder() \
                .addGrid(model_LR_CV.regParam, [0.01, 0.005, 0.0001]) \
                .addGrid(model_LR_CV.elasticNetParam, [0.1, 0.001]) \
                .build()

In [0]:
# create object of the Cross Calidator with 3 folds
cv = CrossValidator(estimator=model_LR_CV,
                    estimatorParamMaps=updated_params,
                    evaluator=evaluator,
                    numFolds=3,
                    seed=27)

In [0]:
# fit the model
grid_model = cv.fit(train_data_encoded_Split)

In [0]:

# Evaluate training data
evaluator = RegressionEvaluator(labelCol="Purchase", metricName="rmse") 
evaluator.evaluate(grid_model.transform(train_data_encoded_Split)) 

Out[72]: 3225.4568580795876

In [0]:
# evaluate model on validation data
evaluator.evaluate(grid_model.transform(valid_data_encoded_Split))

Out[73]: 3206.06739423361

---

Get the best model parameters.

---

In [0]:
# extract the best model parameters dictionary
param_dict = grid_model.bestModel.extractParamMap()

In [0]:
param_dict

Out[75]: {Param(parent='LinearRegression_dfe819337b5d', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2).'): 2,
 Param(parent='LinearRegression_dfe819337b5d', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.001,
 Param(parent='LinearRegression_dfe819337b5d', name='epsilon', doc='The shape parameter to control the amount of robustness. Must be > 1.0. Only valid when loss is huber'): 1.35,
 Param(parent='LinearRegression_dfe819337b5d', name='featuresCol', doc='features column name.'): 'features',
 Param(parent='LinearRegression_dfe819337b5d', name='fitIntercept', doc='whether to fit an intercept term.'): True,
 Param(parent='LinearRegression_dfe819337b5d', name='labelCol', doc='label column name.'): 'Purchase',
 Param(parent='LinearRegression_dfe819337b5d', name='loss', doc='The loss function to be optimized. Supported options: squaredError, huber.'): 'squaredError',
 Param(parent='LinearRegression_dfe819337b5d', name='maxIter', doc='max number of iterations (>= 0).'): 100,
 Param(parent='LinearRegression_dfe819337b5d', name='predictionCol', doc='prediction column name.'): 'prediction',
 Param(parent='LinearRegression_dfe819337b5d', name='regParam', doc='regularization parameter (>= 0).'): 0.0001,
 Param(parent='LinearRegression_dfe819337b5d', name='solver', doc='The solver algorithm for optimization. Supported options: auto, normal, l-bfgs.'): 'auto',
 Param(parent='LinearRegression_dfe819337b5d', name='standardization', doc='whether to standardize the training features before fitting the model.'): True,
 Param(parent='LinearRegression_dfe819337b5d', name='tol', doc='the convergence tolerance for iterative algorithms (>= 0).'): 1e-06}

In [0]:
# created a filtered dictionary
final_dict = {}
for k, v in param_dict.items():
    final_dict[k.name] = v

In [0]:
# get the best elastic net parameter
final_dict["elasticNetParam"]

Out[77]: 0.001

In [0]:
# get the best regularization parameter
final_dict["regParam"]

Out[78]: 0.0001

----

In [0]:
#finding Mode of Product_Category_2 for missing value imputation

Product_Category_2_DF= train_data.select('Product_Category_2')
# Create a view
Product_Category_2_DF.registerTempTable('table')

Product_Category_2_DF=spark.sql(
    'SELECT Product_Category_2, COUNT(Product_Category_2) AS count FROM table GROUP BY Product_Category_2 ORDER BY count desc limit 1'
)
Product_Category_2_DF.show()


+------------------+-----+
Product_Category_2|count|
+------------------+-----+
 2|43534|
+------------------+-----+

In [0]:
#finding Mode of Product_Category_3 for missing value imputation

Product_Category_3_DF= train_data.select('Product_Category_3')
# Create a view
Product_Category_3_DF.registerTempTable('table')
Product_Category_3_DF=spark.sql(
    'SELECT Product_Category_3, COUNT(Product_Category_3) AS count FROM table GROUP BY Product_Category_3 ORDER BY count desc limit 1'
)
Product_Category_3_DF.show()


+------------------+-----+
Product_Category_3|count|
+------------------+-----+
 16|32636|
+------------------+-----+

------

10.Create a Spark ML Pipeline for the final model.

-------

In [0]:
# importing the required libraries
from pyspark.sql import SparkSession
import pyspark.sql.types as tp
from pyspark.sql import functions as F
import pandas as pd

from pyspark.ml.feature import MinMaxScaler

# libraries to make plots
import matplotlib.pyplot as pltA
%matplotlib inline

# create spark session
spark = SparkSession.builder.getOrCreate()


# reading the train data
train_data = spark.read.csv("/FileStore/tables/train_Spark_ML_Assignment.csv",inferSchema=True, header=True)

# reading the test data
test_data  = spark.read.csv("/FileStore/tables/test_Spark_ML_Assignment.csv", inferSchema=True, header=True)

In [0]:
train_data, valid_data = train_data.randomSplit([0.7, 0.3])


In [0]:
## Import the Required Libraries 
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCol, HasOutputCol, Param
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.feature import MinMaxScaler



---
### `Define the stages of the Pipeline`


  * **STAGE 1**: [Transformer] Fill null values with in each column Product_Category_2 , Product_Category_3
    
  * **STAGE 2**: [Estimator] Label Encode: Age
  * **STAGE 3**: [Estimator] Label Encode: Gender
  * **STAGE 4**: [Estimator] Label Encode: Stay_In_Current_City_Years
  * **STAGE 5**: [Estimator] Label Encode: City_Category
    
  * **STAGE 6**: [Estimator] OHE: Gender, City_Category, Occupation, Product_Category_1, Product_Category_2 , Product_Category_3

  * **STAGE 7**: [Transformer] Create Feature: Purchase per Product_ID -- >Mean_Purchase , count of products per Product_ID --> category_count
    
  * **STAGE 8**: [Transformer] VectorAssembler: Mean_Purchase  & Product_Category_count
  
  * **STAGE 9**: [Transformer] MinMaxScaler: Mean_Purchase & Product_Category_count --> FETemp_scaled
      
  * **STAGE 10**: [Transformer] Create Vector [ Age (LE), Stay_In_Current_City_Years,(LE), Gender (OHE), Occupation(OHE), City_Category(OHE), Product_Category_1(OHE), Product_Category_2(OHE) , Product_Category_3(OHE), Marital_Status, FETemp_scaled
                                              
  * **STAGE 11**: [Estimator] Predict Labels Using the Linear Regression

In [0]:
# custom transformer to fill null values

class nullValuesTransformer(Transformer):
    
    def __init__(self, dataframe = None):
        self.dataframe = dataframe
    
    def _transform(self, dataframe):
        dataframe = dataframe.fillna({
            "Product_Category_2" : 2,
            "Product_Category_3": 16,
            
        })
        
        return dataframe

In [0]:

Mean_Purchase_valueDF = train_data.groupby("Product_ID").agg(F.avg("Purchase").alias("Mean_Purchase"))
Prod_CategoryDF = train_data.groupBy("Product_ID").agg(F.count("Product_ID").alias("Product_Category_count"))


In [0]:
# Creating two new features:
# Purchase per Product_ID -- >Mean_Purchase ,
# count of products per Product_ID --> category_count

class FeatEngg(Transformer):
    
    def __init__(self, dataframe = None):
        self.dataframe = dataframe
        
    def _transform(self, dataframe):
        
        
        # join  Purchase per Product_ID 
        dataframe = dataframe.join(Mean_Purchase_valueDF, ['Product_ID'], "inner")
        
        # join count of products per Product_ID 
        dataframe = dataframe.join(Prod_CategoryDF, ['Product_ID'], "inner")
        
        # replace null values
        dataframe = dataframe.fillna({
            'Mean_Purchase': 0.0,
            'Product_Category_count' : 0.0,
        })
        
        return dataframe
        

In [0]:
# Stage 1 - replace null values
stage_1 = nullValuesTransformer()

# stage 2 - Label Encode: Age
stage_2 =  StringIndexer(inputCol= "Age", outputCol= "Age_le" , handleInvalid="skip")

# Stage 3 -  Label Encode: Gender
stage_3 = StringIndexer(inputCol= "Gender", outputCol= "Gender_le", handleInvalid= "skip")

# Stage 4 - Label Encode: Stay_In_Current_City_Years
stage_4 = StringIndexer(inputCol= "Stay_In_Current_City_Years", outputCol= "Stay_In_Current_City_Years_le", handleInvalid= "skip")

# Stage 5 - Label Encode: City_Category
stage_5 = StringIndexer(inputCol= "City_Category", outputCol= "City_Category_le", handleInvalid="skip")

# Stage 6 - OHE: Gender, City_Category, Occupation, Product_Category_1, Product_Category_2 , Product_Category_3

stage_6 =  OneHotEncoder(inputCols=["Gender_le",
                                     "City_Category_le",
                                      "Occupation", 'Product_Category_1', 'Product_Category_2' , 'Product_Category_3'],
                          outputCols=["Gender_ohe",
                                      "City_Category_ohe",
                                      "Occupation_ohe", 'Product_Category_1_ohe', 'Product_Category_2_ohe','Product_Category_3_ohe'])

# stage 7 - Create new features 
stage_7 = FeatEngg()

# stage 8 - VectorAssembler - Mean_Purchase
stage_8 = VectorAssembler(inputCols=['Mean_Purchase', 'Product_Category_count'],
                          
                          outputCol = 'FETemp')


# # Stage 9 -  MinMaxScaler: Mean_Purchase 
stage_9 =  MinMaxScaler(inputCol='FETemp', outputCol='FETemp_scaled')
 

# Stage 10 - Create vector from the columns
stage_10 = VectorAssembler(inputCols= ['Age_le','Stay_In_Current_City_Years_le',
                                       'Gender_ohe', 'City_Category_ohe','Occupation_ohe',
                                       'Product_Category_1_ohe', 'Product_Category_2_ohe','Product_Category_3_ohe',
                                      'Marital_Status','FETemp_scaled'],

                         outputCol=  "feature_vector")

# Stage 11 - Train ML model
stage_11 = LinearRegression(featuresCol= "feature_vector", labelCol= "Purchase", maxIter=20, regParam=0.0001, elasticNetParam=0.001)



In [0]:
# Define pipeline
pipeline = Pipeline(stages= [stage_1,
                             stage_2,
                             stage_3,
                             stage_4,
                             stage_5,
                             stage_6,
                             stage_7,
                             stage_8,
                             stage_9,
                             stage_10,
                             stage_11
                            ])

In [0]:
# fit the pipeline with the training data
pipeline_model = pipeline.fit(train_data)


In [0]:
# transform data
final_train_data = pipeline_model.transform(train_data)


# TRANSFORM THE VALIDATION DATA
predictions_valid_data  = pipeline_model.transform(valid_data)


# TRANSFORM THE TEST DATA
predictions_test_data = pipeline_model.transform(test_data)

In [0]:
# Evaluate training data
evaluator = RegressionEvaluator(labelCol="Purchase", metricName="rmse") 

# evaluate model on train data
evaluator.evaluate(final_train_data)


Out[91]: 2644.495346939429

In [0]:

trainingSummary = pipeline_model.stages[-1].summary

print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 2644.495347
r2: 0.722714

In [0]:
# evaluate model on validation data
evaluator.evaluate(predictions_valid_data)


Out[93]: 2671.235557742257

In [0]:
predictions_valid_data.select("feature_vector","Purchase","prediction").show()



+--------------------+--------+------------------+
 feature_vector|Purchase| prediction|
+--------------------+--------+------------------+
(84,[0,1,15,28,49...| 13645| 8306.309476919543|
(84,[0,1,15,27,49...| 12842|13446.402409678509|
(84,[0,1,2,4,21,2...| 19684|16310.609195585046|
(84,[1,2,20,26,50...| 12103|10305.125696369623|
(84,[0,1,2,3,12,2...| 15686| 15622.66110978646|
(84,[2,33,47,79,8...| 5918| 8734.132622394978|
(84,[2,33,47,79,8...| 6073| 7662.858362367854|
(84,[2,26,47,68,8...| 15665|14047.717084404721|
(84,[2,26,47,79,8...| 11603|12952.076166339923|
(84,[2,33,47,79,8...| 4163|5903.9494343599445|
(84,[0,14,27,48,6...| 13055| 10242.28844405978|
(84,[1,2,4,17,33,...| 7875| 8175.906440749615|
(84,[1,2,4,17,33,...| 7869|7901.5683363719245|
(84,[1,2,4,17,33,...| 7820| 7229.345767938066|
(84,[1,2,4,17,33,...| 9970| 7977.139035734622|
(84,[1,2,4,17,33,...| 5965|8078.6938635630695|
(84,[1,2,4,17,27,...| 15941|13529.431088164294|
(84,[1,2,4,17,30,...| 5307| 6232.929607747065|
(84,[1,2,4,17,33,...| 9872| 8591.88468760279|
(84,[1,2,4,17,33,...| 9769| 7607.875991956549|
+--------------------+--------+------------------+
only showing top 20 rows

---- End ---